In [ ]:
import os
# 자바 설치후 자바가 설치된 폴더 설정
os.environ["JAVA_HOME"] = "C:/bigdata/jdk-18.0.2/"

In [ ]:
import sys
# 자바의 실행 파일 java.exe가 존재하는 경로를 path에 추가
sys.path.append("C:/bigdata/jdk-18.0.2/bin/java.exe")

In [ ]:
# 한글 형태소/단어 분리를 위한 라이브러리 설치
!pip install konlpy

In [ ]:
# 형태소/단어 분리에 도움을 주는 라이브러리 설치
!pip install Twitter

In [ ]:
import pandas as pd

In [ ]:
# 파일에 저장되어 있는 데이터를 불러옴
youtube = pd.read_csv("환경부_웹_크롤링.csv")

In [ ]:
import matplotlib.pyplot as plt
plt.style.use ('ggplot')

In [ ]:
import seaborn as sns
# 이상치 줄이기 전 그래프
sns.distplot(youtube['view_num'])

In [ ]:
import numpy as np

# 유튜브 조회수 데이터에 최근에 올라온 영상이나 특이하게 많이 시청된 영상을 이상치로 판단하고 제거
def outliers(data):
    # Q!, Q3, IQR을 설정
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    
    # 설정한 범위를 넘어가면 이상치라 판단
    return np.where((data > upper_bound)|(data < lower_bound))

In [ ]:
# outliers 함수에 유튜브 조회수 데이터를 넣고 이상치 판단
outlier_index = outliers(youtube['view_num'])[0]

In [ ]:
# 이상치가 아닌 데이터를 넣은 리스트를 만듦
not_outlier = []
# 유튜브 조회수 데이터에서 이상치가 아닌 데이터가 있으면 위의 리스트에 넣음
for i in youtube.index:
    if i not in outlier_index:
        not_outlier.append(i)

In [ ]:
# youtube 데이터에서 이상치가 아닌 데이터를 지정
youtube = youtube.loc[not_outlier]
# 그 외의 것들은 drop하고 인덱스를 재설정함
youtube = youtube.reset_index(drop = True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# 이상치 제거 후 그래프
sns.distplot(youtube['view_num'])

In [ ]:
from konlpy.tag import Twitter

In [ ]:
twitter = Twitter()

In [ ]:
# 텍스트를 전처리 하는 함수를 만듦
def preprocessingText(text):
    # 전처리한 결과를 저장할 리스트
    stems = []
    # text를 단어로 나누고 품사를 저장
    # stem = True: 단어의 기본형으로 변환
    tagged_review = twitter.pos(text, stem = True)
    
    # 단어는 word, 품사는 pos에 대입
    for word, pos in tagged_review:
        # 품사가 명사, 형용사, 동사, 부사이면 stems에 추가
        if pos == "Noun" or pos == "Adjective" or pos =="Verb" or pos == "Adverb":
            stems.append(word)
    
    # 리스트인 stems를 문자열로 만들어 리턴
    return " ".join(stems)

In [ ]:
# 유튜브 제목을 데이터 처리하여 넣을 열을 추가하고 초기값을 결측치로 설정
import numpy as np
youtube["title_convert"] = np.NaN

In [ ]:
from tqdm.notebook import tqdm

# title 행의 개수만큼 반복
for index in tqdm(range(len(youtube['title']))):
    try:
        # title 행의 데이터를 title_convert에 대입하고, 함수를 사용하여 단어 리턴
        title = youtube.loc[index, "title"]
        youtube.loc[index, "title_convert"] = preprocessingText(title)
        
    except Exception as e:
        # 에러가 발생했을 때 결측치 대입
        youtube.loc[index, "title_convert"] = np.NaN

In [ ]:
# title_convert에 글자수가 1 이상인 행만 저장(한글이 아닌 모든 데이터를 삭제 - 영어, 중국어, 이모티콘 등)
youtube = youtube[youtube["title_convert"].str.len() > 1]

In [ ]:
# 중간중간에 index 숫자가 빠져있으므로, 기존 index 제거
youtube.reset_index(drop = True, inplace = True)

In [ ]:
# title_convert_list 열을 만들어서 title_convert의 데이터를 공백을 기준으로 단어별로 분리해서 리턴
youtube["title_convert_list"] = youtube["title_convert"].apply(lambda data :data.split(" "))

In [ ]:
# 1, 2, 3 분위수, IQR, 평균 지정
Q1 = youtube['view_num'].quantile(.25)
Q2 = youtube['view_num'].quantile(.5)
Q3 = youtube['view_num'].quantile(.75)
IQR = Q3 - Q1
mean = youtube['view_num'].mean()
max = youtube['view_num'].max()
min = youtube['view_num'].min()


In [ ]:
view_num_list = []

# youtube 데이터의 행의 수만큼 반복
for index in range(len(youtube)):
    view_num = youtube['view_num'][index]
    # 1분위수보다 작다면 0으로 치환 후 리스트에 저장
    if(view_num < mean):
        view_num_list.append(0)

    else:
        view_num_list.append(1)

In [ ]:
# 새로운 열에 리스트 대입
youtube["view_num_convert"] = view_num_list

In [ ]:
from sklearn.model_selection import train_test_split

# 전체 데이터중 80% 는 학습 데이터, 20%는 테스트 데이터로 추출
X_train, X_test, y_train, y_test = train_test_split(
youtube['title_convert_list'], # 독립변수
youtube['view_num_convert'], # 종속변수
test_size = 0.2, # 테스트 사이즈 지정
stratify = youtube['view_num_convert'], # 데이터 분리시 0, 1, 2, 3의 빈도수 비율 그대로 분리
random_state = 1 # random_state를 지정해주지 않으면 실행할때마다 다른 결과가 나옴
)

In [ ]:
# 조회수 높은 영상에서 자주 사용되는 단어 빈도를 찾아보기 위해 조회수 높은 영상만 출력
under = youtube[youtube["view_num_convert"] == 0].index
youtube_over = youtube.drop(under)

In [ ]:
# 조회수 높은 영상에서 사용된 모든 단어 출력
for line in youtube_over['title_convert_list']:
    print(" ".join(line))

In [ ]:
from konlpy.tag import Okt
from collections import Counter

In [ ]:
# 위의 전체 내용에서 단어가 한번 발생하면 리스트에 추가해주고, 추가적으로 발생하면 원래 단어에 더해주는 함수 구현
wordCount = {}
for line in youtube_over['title_convert_list']:
    line_str = " ".join(line)

    wordList = line_str.split()

 
    for word in wordList:
        wordCount[word] = int(wordCount.get(word, 0) + 1 )
        keys = sorted(wordCount.keys())
    

In [ ]:
wordCount

In [ ]:
# 최다 빈도수 단어부터 시작해서 내림차순으로 정렬
sort_count = sorted(wordCount.items(), key = lambda x:x[1], reverse=True)
# 시각화를 위해 제일 많이 사용된 단어 25개를 가져옴
converted_dict = dict(sort_count[:25])
print(converted_dict)

In [ ]:
# 조회수가 많은 영상에서 최다 빈도 단어를 바 그래프로 그림
import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'

matplotlib.rcParams['axes.unicode_minus'] =False

keys = converted_dict.keys()
values = converted_dict.values()

parameters = {
              'axes.labelsize': 50,
              'axes.titlesize': 35,
              'xtick.labelsize': 40,
              'ytick.labelsize': 40
              }
plt.rcParams.update(parameters)


plt.figure(figsize = (60,30))
plt.bar(keys, values)

In [ ]:
# 제목과 조회수를 전처리한 데이터 외의 다른 내용들은 삭제
youtube = youtube.drop(['view_num'], axis = 1)
youtube = youtube.drop(['title'], axis = 1)
youtube = youtube.drop(['title_convert'], axis = 1)

In [ ]:
# 데이터프레임을 파일로 저장
youtube.to_csv("환경부_데이터_전처리.csv", index = False)

In [ ]:
!pip install gensim

In [ ]:
from gensim.models.word2vec import Word2Vec

# 각 단어들을 학습해서 Vector 형태로 생성
word2vec = Word2Vec(X_train, # 리스트 형태의 X_train 데이터
                   sg = 2, # word2vec 의 skip-gram 형식 사용
                   window = 5, # 고려할 앞뒤 폭 (앞뒤 5개 단어)
                   min_count = 2, # 2회 이하 사용된 단어 무시)
                    workers = 10 # 동시에 처리학 작업 수
                   )

In [ ]:
# 워드 벡터의 전체 단어 수와 각 벡터의 크기를 저장
num_words, emb_dim = word2vec.wv.vectors.shape

In [ ]:
# w2cSorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=True))

In [ ]:
# !pip install protobuf==3.20

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, LSTM, Bidirectional, Dense, Embedding, Flatten, SimpleRNN

In [ ]:
# 인덱스를 입력하면 해당 인덱스의 단어를 벡터 값으로 변환하는 객체
emb = Embedding(input_dim = num_words, output_dim = emb_dim,
               trainable = False, weights = [word2vec.wv.vectors])

In [ ]:
# 첫번째 단어를 벡터화 한 데이터
word2vec.wv.vectors[0]

In [ ]:
# 선형회귀를 진행하는 모델 생성 후, 위의 객체 추가
seq = Sequential()
seq.add(emb)

In [ ]:
# word2vec에 저장된 단어들을 Dictionary 객체로 리턴
word_vector_keys = list(word2vec.wv.key_to_index.keys())

In [ ]:
# X_train에 저장된 문자열을 인덱스로 변환해서 저장할 배열
X_train_int= []

# X_train의 각 행을 column에 저장
for column in tqdm(X_train):
    input = []
    
    # 단어가 Dictionary에 저장되어 있으면 input에 추가
    for word in column:
        if word in word_vector_keys:
            input.append(word2vec.wv.index_to_key.index(word))

    X_train_int.append(input)

In [ ]:
# X_test에 저장된 문자열을 인덱스로 변환하여 저장할 배열
X_test_int= []

# X_test의 각 행을 column에 저장
for column in tqdm(X_test):
    input = []
    
    # 단어가 Dictionary에 저장되어 있으면 input에 추가
    for word in column:
        if word in word_vector_keys:
            input.append(word2vec.wv.index_to_key.index(word))

    X_test_int.append(input)

In [ ]:
from tensorflow.keras.preprocessing import sequence

"""각 제목의 길이가 다름.
딥러닝을 하기 위해서는 길이를 통일할 필요가 있으므로, 단어 20글자를 기준으로 잡아 데이터를 채우고, 빈 영역은 0으로 채움
만약 제목 단어의 수가 20보다 길다면, 뒤에서부터 단어 20개 잘라서 맞춤(단어 개수가 20개 넘는 제목이 거의 없음)
"""
# train, test 에서 둘다 실행
X_train_pad = sequence.pad_sequences(X_train_int, maxlen = 20)
X_test_pad = sequence.pad_sequences(X_test_int, maxlen = 20)

In [ ]:
# 조회수를 0, 1으로 변환한 객체의 개수들
count_0 = youtube["view_num_convert"].value_counts()[0]
count_1 = youtube["view_num_convert"].value_counts()[1]

# 전체 데이터의 갯수
total = len(youtube)

In [ ]:
# 각 레벨의 가중치 설정
# 전체 데이터 수 (각 데이터의 개수 * 레벨 종류의 합)
weight_for_0 = total / (count_0 * 2)
weight_for_1 = total / (count_1 * 2)

# 가중치 설정한 것을 class_weight으로 지정
class_weight = {
                    0: weight_for_0,
                    1: weight_for_1
                }

In [ ]:
# 워드벡터 형식으로 바꾼 데이터를 선형회귀하는 모델을 만듦
# 양뱡향 LSTM과 Dense layer를 통해 선형회귀를 진행
# 0, 1의 결과로 선형회귀 하기 때문에, 마지막 layer 로 sigmoid를 사용
model = Sequential()

model.add(Embedding(input_dim = num_words, output_dim = emb_dim,
                   trainable = False, weights = [word2vec.wv.vectors]))
model.add(Bidirectional(LSTM(512, recurrent_dropout = 0.2, return_sequences = True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(256, recurrent_dropout = 0.2, return_sequences = True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128, recurrent_dropout = 0.2, return_sequences = True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64, recurrent_dropout = 0.2, return_sequences = True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64, recurrent_dropout = 0.2)))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))


In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

# 모델을 학습에 적용시킴
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate = 1e-3),
             metrics = ['acc'])

In [ ]:
# 모델을 적합할 때 시간을 단축시키기 위해 형식을 바꿔줌
y_train = np.array(y_train, dtype = "float32")
y_test = np.array(y_test, dtype = "float32")

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# 학습시 가장 정확도가 높은 모델을 저장하도록 설정
cb_checkpoint = ModelCheckpoint(filepath="C:/bigdata/natual_language_processing_1.h5", # 학습 진행시 가장 정확도가 높은 모델을 저장할 경로
                                monitor='loss', # 저장할 조건 val_acc (테스트 데이터의 loss) 가 가장 낮은 모델을 저장
                                vervose=1, #함수의 진행 과정 출력
                                save_best_only=True # 가장 정확도가 높은 모델 1개만 저장
                                )

In [ ]:
# 학습을 이어서 할때, 저장된 weight를 불러옴
model.load_weights("C:/bigdata/natual_language_processing_1.h5")

In [ ]:
hist = model.fit(X_train_pad, y_train, #X_train_pad, y_train 을 통해 학습
                 batch_size = 1000,
                 epochs = 100,
                 class_weight = class_weight, # 지정했던 weight 사용
                 callbacks=[cb_checkpoint], # 학습시 가장 loss가 낮은 모델을 저장하도록 설정 
                )


In [ ]:
# train_test_split으로 만들었던 테스트 데이터로 검사
model.evaluate(X_test_pad, y_test)

In [ ]:
# 유튜브 제목을 임의로 정해서 테스트
my_test_data = [
    '''
    환경부와 함께하는 환경의 날 현장 이야기
    '''
]

In [ ]:
# 새로 만든 제목 또한 워드 벡터 형식으로 바꿔서 테스트 실행
test_data_list = []

for my_data in my_test_data:
    text = preprocessingText(my_data)
    test_data_list.append(text)

In [ ]:
# 새롭게 만든 테스트 데이터를 새로운 리스트에 넣음
my_test_token = []
for column in test_data_list:
    token_to_index = []
    for word in column.split():
        if word in word2vec.wv.index_to_key:
            word_index = word2vec.wv.index_to_key.index(word)
            token_to_index.append(word_index)
            
    my_test_token.append(token_to_index)

In [ ]:
# 새로운 테스트 데이터 또한 같은 형식으로 맞춰줌
my_test_pad = sequence.pad_sequences(my_test_token, maxlen = 20)

In [ ]:
# 모델을 통해 새로운 테스트 데이터를 검사하고, 조회수가 평균보다 높은지 낮은지 판단
predict = model.predict(my_test_pad)
if predict < 0.5:
    print("조회수 평균보다 낮을 것으로 예상")
else:
    print("조회수 평균보다 높을 것으로 예상")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label = 'train loss')


acc_ax.plot(hist.history['acc'], 'b', label = 'train acc')


loss_ax.set_xlabel('epochs')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('acc')

loss_ax.set_ylim([0.69, .70])
acc_ax.set_ylim([0.0, 1.0])

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')


plt.show()